In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-27,199,0.0
72308,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-28,199,0.0
72309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-29,199,0.0
72310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-10-30,199,0.0


In [3]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32226, 674)
y_train.shape=(32226,)
X_test.shape=(40086, 674)
y_test.shape=(40086,)


In [6]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [7]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2023-01-04 13:14:19,000] A new study created in memory with name: no-name-8f899f10-6f6f-4c6b-a6bd-c9935e37d3d6


[LightGBM] [Warning] bagging_fraction is set=0.509600620422143, subsample=1.0 will be ignored. Current value: bagging_fraction=0.509600620422143
[LightGBM] [Warning] feature_fraction is set=0.735691412359843, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.735691412359843


[I 2023-01-04 13:16:02,328] Trial 0 finished with value: 0.4852752508170715 and parameters: {'num_leaves': 105, 'feature_fraction': 0.735691412359843, 'bagging_fraction': 0.509600620422143, 'min_child_samples': 41}. Best is trial 0 with value: 0.4852752508170715.


[LightGBM] [Warning] bagging_fraction is set=0.5464189359948948, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5464189359948948
[LightGBM] [Warning] feature_fraction is set=0.5868480384232153, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5868480384232153
[LightGBM] [Warning] bagging_fraction is set=0.5464189359948948, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5464189359948948
[LightGBM] [Warning] feature_fraction is set=0.5868480384232153, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5868480384232153


[I 2023-01-04 13:16:25,435] Trial 1 finished with value: 0.4580109922962035 and parameters: {'num_leaves': 156, 'feature_fraction': 0.5868480384232153, 'bagging_fraction': 0.5464189359948948, 'min_child_samples': 26}. Best is trial 1 with value: 0.4580109922962035.


...

[I 2023-01-04 13:16:37,223] Trial 2 finished with value: 0.5484434725933172 and parameters: {'num_leaves': 51, 'feature_fraction': 0.7670872914267415, 'bagging_fraction': 0.9257726290857218, 'min_child_samples': 15}. Best is trial 1 with value: 0.4580109922962035.


[LightGBM] [Warning] bagging_fraction is set=0.43841255770739274, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43841255770739274
[LightGBM] [Warning] feature_fraction is set=0.5379987902487102, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5379987902487102
[LightGBM] [Warning] bagging_fraction is set=0.43841255770739274, subsample=1.0 will be ignored. Current value: bagging_fraction=0.43841255770739274
[LightGBM] [Warning] feature_fraction is set=0.5379987902487102, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5379987902487102


[I 2023-01-04 13:17:14,883] Trial 3 finished with value: 0.46129337308423635 and parameters: {'num_leaves': 142, 'feature_fraction': 0.5379987902487102, 'bagging_fraction': 0.43841255770739274, 'min_child_samples': 6}. Best is trial 1 with value: 0.4580109922962035.


[LightGBM] [Warning] bagging_fraction is set=0.625309822442107, subsample=1.0 will be ignored. Current value: bagging_fraction=0.625309822442107
[LightGBM] [Warning] feature_fraction is set=0.6402569735940813, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6402569735940813
[LightGBM] [Warning] bagging_fraction is set=0.625309822442107, subsample=1.0 will be ignored. Current value: bagging_fraction=0.625309822442107
[LightGBM] [Warning] feature_fraction is set=0.6402569735940813, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6402569735940813


[I 2023-01-04 13:18:44,016] Trial 4 finished with value: 0.4349796101389788 and parameters: {'num_leaves': 161, 'feature_fraction': 0.6402569735940813, 'bagging_fraction': 0.625309822442107, 'min_child_samples': 34}. Best is trial 4 with value: 0.4349796101389788.


In [8]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 161, 'feature_fraction': 0.6402569735940813, 'bagging_fraction': 0.625309822442107, 'min_child_samples': 34}


In [9]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.625309822442107, subsample=1.0 will be ignored. Current value: bagging_fraction=0.625309822442107
[LightGBM] [Warning] feature_fraction is set=0.6402569735940813, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6402569735940813


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x130373b80>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.625309822442107,
                               feature_fraction=0.6402569735940813,
                               min_child_samples=34, num_leaves=161))])

In [10]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

test_mae=2.4575


In [11]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=2979,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [12]:
plot_one_sample(
    example_id=3979,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)